In [50]:
import sys
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import talib
import torch.utils.data as Data
sys.path.append('../../')
import  DataSource
zz500 = DataSource.get_zz500_codes() # 我看中证500的数据
# 我这里查看第一个股票的吧
N = 60 # 多少日的数据
# 我这里仅仅是记录如下的列的涨跌幅度
rate_columns = ['open', 'high', 'low', 'close']
close_index = rate_columns.index('close')
input_size = len(rate_columns)
seq_len = N    # 多少个时间序列的股票
output_size = 1 # 我只是输出一个今天的收盘价相比较昨日的涨跌比率
num_layers = 10 # 多少个gru合并
hidden_size = 20 # 隐藏层的宽度
batch_size = 16 # 一个批次有多少个数据
lr = 0.0001

In [51]:
dt = DataSource.get_data('sh.600004')
dt.head()

,code,open,high,low,close,preclose,volume,amount,adjustflag,turn,tradestatus,pctChg,isST
date,,,,,,,,,,,,,
2010-01-04,sh.600004,5.046509,5.111144,5.041537,5.046509,5.046509,15478854,157747809.0,2,1.525010,1,0.0000,0
2010-01-05,sh.600004,5.046509,5.091256,5.041537,5.066397,5.046509,11070922,112683563.0,2,1.090731,1,0.3941,0
2010-01-06,sh.600004,5.056453,5.066397,4.981874,4.991818,5.066397,9410665,95205863.0,2,0.927159,1,-1.4720,0
2010-01-07,sh.600004,4.971930,4.996790,4.827744,4.862548,4.991818,8961813,88611912.0,2,0.882937,1,-2.5896,0
2010-01-08,sh.600004,4.837688,5.096228,4.837688,5.061425,4.862548,14923812,149969456.0,2,1.470326,1,4.0900,0


In [52]:
dt_list = []
for j in range(N+1):
    # N+1表示要多一天，比如前面30天，我要看看第31天的收盘价相对于第一天的收盘价是什么比例
    for i in rate_columns:
        dt_tmp = pd.DataFrame({f'{i}_rate_{j}':(dt[f'{i}'].shift(-j) - dt[f'preclose'])/dt[f'preclose']})
        dt_list.append(dt_tmp)

dt2 = pd.concat(dt_list[:-(len(rate_columns))], axis=1)
dt2.head()

,open_rate_0,high_rate_0,low_rate_0,close_rate_0,open_rate_1,high_rate_1,low_rate_1,close_rate_1,open_rate_2,high_rate_2,...,low_rate_57,close_rate_57,open_rate_58,high_rate_58,low_rate_58,close_rate_58,open_rate_59,high_rate_59,low_rate_59,close_rate_59
date,,,,,,,,,,,,,,,,,,,,,
2010-01-04,0.000000,0.012808,-0.000985,0.000000,0.000000,0.008867,-0.000985,0.003941,0.001970,0.003941,...,0.115271,0.121182,0.119212,0.135961,0.117241,0.134975,0.133005,0.151724,0.129064,0.150739
2010-01-05,0.000000,0.008867,-0.000985,0.003941,0.001970,0.003941,-0.012808,-0.010837,-0.014778,-0.009852,...,0.117241,0.134975,0.133005,0.151724,0.129064,0.150739,0.151724,0.158621,0.139901,0.157635
2010-01-06,-0.001963,0.000000,-0.016683,-0.014720,-0.018646,-0.013739,-0.047105,-0.040236,-0.045142,0.005888,...,0.124632,0.146222,0.147203,0.154073,0.135427,0.153091,0.151129,0.151129,0.132483,0.141315
2010-01-07,-0.003984,0.000996,-0.032869,-0.025896,-0.030876,0.020916,-0.030876,0.013944,0.021912,0.041833,...,0.152390,0.170319,0.168327,0.168327,0.149402,0.158367,0.157371,0.159363,0.137450,0.139442
2010-01-08,-0.005112,0.048057,-0.005112,0.040900,0.049080,0.069530,0.040900,0.062372,0.056237,0.070552,...,0.179959,0.189162,0.188139,0.190184,0.167689,0.169734,0.173824,0.188139,0.170757,0.186094


In [53]:
-(len(rate_columns)- close_index)

-1

In [54]:
dt_list[-1].head()

,close_rate_60
date,
2010-01-04,0.157635
2010-01-05,0.145813
2010-01-06,0.122669
2010-01-07,0.155378
2010-01-08,0.218814


In [55]:
y_close = dt_list[-(len(rate_columns)- close_index)]
y_close.head()

,close_rate_60
date,
2010-01-04,0.157635
2010-01-05,0.145813
2010-01-06,0.122669
2010-01-07,0.155378
2010-01-08,0.218814


In [56]:
x_price = dt2.to_numpy()
x_price = x_price[:-(N+1)]
# 更改维度
x_shape_old = x_price.shape
x_shape_new_1 = x_shape_old[0]
x_shape_new_3 = len(rate_columns)
x_shape_new_2 = int(x_shape_old[1] / x_shape_new_3)
x_price = x_price.reshape((x_shape_new_1, x_shape_new_2, x_shape_new_3))
x_price[0]

array([[ 0.        ,  0.01280788, -0.00098522,  0.        ],
       [ 0.        ,  0.008867  , -0.00098522,  0.00394089],
       [ 0.00197044,  0.00394089, -0.01280788, -0.01083744],
       [-0.01477833, -0.00985222, -0.04334975, -0.0364532 ],
       [-0.04137931,  0.00985222, -0.04137931,  0.00295567],
       [ 0.01083744,  0.03054187,  0.00295567,  0.02364532],
       [ 0.01773399,  0.03152709,  0.00985222,  0.03054187],
       [ 0.01083744,  0.03448276,  0.00591133,  0.01576355],
       [ 0.01477833,  0.05714286,  0.01280788,  0.05221675],
       [ 0.05123153,  0.05714286,  0.03743842,  0.04433498],
       [ 0.04433498,  0.09162562,  0.03448276,  0.07980296],
       [ 0.07389163,  0.10344828,  0.06896552,  0.09359606],
       [ 0.0955665 ,  0.09950739,  0.0729064 ,  0.08275862],
       [ 0.10837438,  0.1182266 ,  0.06403941,  0.0679803 ],
       [ 0.07389163,  0.1408867 ,  0.06502463,  0.09162562],
       [ 0.09458128,  0.11133005,  0.04334975,  0.0453202 ],
       [ 0.0453202 ,  0.

In [57]:
x_price.shape

(3602, 60, 4)

In [58]:
y_close = dt_list[-(len(rate_columns)- close_index)][f'close_rate_{N}'].to_numpy()[:-(N+1)]
y_close

array([ 0.15763547,  0.14581281,  0.12266928, ..., -0.03501545,
       -0.03099174, -0.04145078])

In [59]:
x_price = dt2.to_numpy()
x_price = x_price[:-(N+1)]
# 更改维度
x_shape_old = x_price.shape
x_shape_new_1 = x_shape_old[0]
x_shape_new_3 = len(rate_columns)
x_shape_new_2 = int(x_shape_old[1] / x_shape_new_3)
x_price = x_price.reshape((x_shape_new_1, x_shape_new_2, x_shape_new_3))
# y
y_close = dt_list[-(len(rate_columns)- close_index)][f'close_rate_{N}'].to_numpy()[:-(N+1)]
# 然后是批次的整数倍
_x_y_len = int(x_price.shape[0]/batch_size) * batch_size
print(len(x_price) - _x_y_len)

x_price = x_price[-_x_y_len:]
y_close = y_close[-_x_y_len:]
x_price = x_price.reshape(x_price.shape[0]//batch_size, batch_size, x_price.shape[1], x_price.shape[2])
y_close = y_close.reshape(y_close.shape[0]//batch_size, batch_size, 1)

2


In [60]:
x_price.shape

(225, 16, 60, 4)

In [61]:
y_close.shape

(225, 16, 1)

In [66]:
# 看起来像是正常啊
x_price[0, :,-1,3]

array([0.14131501, 0.13944223, 0.18609407, 0.17092338, 0.14629451,
       0.17112811, 0.17652764, 0.11329588, 0.06415094, 0.01277372,
       0.01801802, 0.00909918, 0.01660517, 0.00722022, 0.01131008,
       0.01347449])

In [74]:
y_close[0]

array([[ 0.12266928],
       [ 0.15537849],
       [ 0.21881391],
       [ 0.16994106],
       [ 0.17901829],
       [ 0.15965583],
       [ 0.15324927],
       [ 0.05617978],
       [ 0.04716981],
       [ 0.0310219 ],
       [-0.0009009 ],
       [ 0.00272975],
       [ 0.0295203 ],
       [-0.03158845],
       [-0.00754006],
       [ 0.00384986]])